### Automated News Data Pipeline: Targeted Categorization

In [ ]:
# Block 1: WorldWatch AI - Focused Categorical Pipeline (V15.0 - Corrected)
# Purpose: This script uses targeted keyword searches to collect news ONLY for
# specific categories (War, Peace, Economic, Trade, Tech). Irrelevant news is never fetched.
# It runs for a set duration and then stops automatically.

# --- 1. SETUP ---
import requests
import pandas as pd
import time
import os
from datetime import datetime
from IPython.display import clear_output

# --- 2. CONFIGURATION ---

# <<< MODIFICATION: Replaced broad endpoints with specific, targeted keyword queries.
# This is the core of the correction. We now ONLY ask the API for relevant news.
CATEGORIZED_QUERIES = {
    "War": "war OR conflict OR invasion OR military action OR airstrike OR geopolitics",
    "Peace": "peace talks OR treaty OR diplomacy OR ceasefire OR negotiations",
    "Economic": "economy OR inflation OR recession OR interest rates OR gdp OR market crash",
    "Trade": "trade deal OR tariff OR supply chain OR exports OR imports",
    "Tech": "AI OR artificial intelligence OR semiconductor OR cybersecurity OR tech giant"
}

MY_API_KEY = "1ddd3a86143c475cb371f487a3617cbc" # Your NewsAPI key
DATASET_FILE = 'News_2025.csv'
UPDATE_INTERVAL_SECONDS = 10
RUN_DURATION_SECONDS = 30

# --- 3. HELPER FUNCTION ---
def fetch_live_news_from_url(api_key, url):
    """Fetches and minimally processes news from a single NewsAPI URL."""
    headers = {'X-Api-Key': api_key}
    try:
        response = requests.get(url, headers=headers, timeout=15)
        if response.status_code == 200:
            articles = response.json().get('articles', [])
            if not articles: return pd.DataFrame()
            df = pd.DataFrame(articles)
            df.rename(columns={'url': 'link', 'urlToImage': 'image'}, inplace=True)
            df['source'] = df['source'].apply(lambda s: s.get('name') if isinstance(s, dict) else s)
            # We now also want 'description' for context, and the new 'category' column will be added later.
            required_cols = ['title', 'source', 'publishedAt', 'description', 'link', 'image']

            # Ensure all columns exist to prevent errors, even if empty.
            for col in required_cols:
                if col not in df.columns:
                    df[col] = ''

            df = df.dropna(subset=['title', 'publishedAt'])
            return df[required_cols]
        else:
            print(f"   ⚠️ API Error (Status: {response.status_code})")
            return None
    except requests.exceptions.RequestException as e:
        print(f"   ❌ Network Error: {e}")
        return None

# --- 4. THE PIPELINE FUNCTION ---
def run_data_pipeline():
    """
    This function runs for a set duration, fetching news ONLY for the predefined
    categories and saving them to the master CSV file.
    """
    start_time = time.time()
    cycle_count = 0

    try:
        while time.time() - start_time < RUN_DURATION_SECONDS:
            elapsed_time = time.time() - start_time
            cycle_count += 1
            clear_output(wait=True)
            print(f"--- [ WorldWatch AI Focused Pipeline | V15.0 ] ---")
            print(f"Running for {RUN_DURATION_SECONDS}s. Time elapsed: {int(elapsed_time)}s")
            print(f"Starting Cycle {cycle_count} at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            print("-" * 50)

            # === Step 1: Load Existing Dataset ===
            try:
                if os.path.exists(DATASET_FILE):
                    local_df = pd.read_csv(DATASET_FILE)
                    existing_titles = set(local_df['title']) if 'title' in local_df.columns else set()
                    print(f"✅ Loaded local dataset '{DATASET_FILE}' with {len(local_df)} articles.")
                else:
                    local_df = pd.DataFrame()
                    existing_titles = set()
                    print(f"ⓘ Local dataset not found. A new '{DATASET_FILE}' will be created.")
            except Exception as e:
                print(f"❌ CRITICAL ERROR: Could not read local dataset. Error: {e}")
                time.sleep(UPDATE_INTERVAL_SECONDS)
                continue

            # === Step 2: Fetch Live News by PREDEFINED CATEGORY ===
            # <<< MODIFICATION: The fetching logic is completely new and targeted.
            print("\n🔎 Fetching targeted news from API...")
            live_articles_dfs = []
            for category, query in CATEGORIZED_QUERIES.items():
                print(f"   Querying for: '{category}'")
                # Build the specific URL for the query using the powerful /everything endpoint
                url = f"https://newsapi.org/v2/everything?q=({query})&language=en&sortBy=publishedAt"

                new_df = fetch_live_news_from_url(MY_API_KEY, url)

                if new_df is not None and not new_df.empty:
                    # IMPORTANT: We add the 'category' tag to the data we just fetched.
                    new_df['category'] = category
                    live_articles_dfs.append(new_df)
                time.sleep(1) # Be polite to the API server

            # === Step 3: Identify NEW Articles ===
            newly_added_articles = []
            if live_articles_dfs:
                live_df_combined = pd.concat(live_articles_dfs, ignore_index=True)
                live_df_combined.drop_duplicates(subset=['title'], keep='first', inplace=True)

                for _, article in live_df_combined.iterrows():
                    if article['title'] not in existing_titles:
                        newly_added_articles.append(article)

            # === Step 4: Report Findings and Save ===
            if newly_added_articles:
                new_articles_df = pd.DataFrame(newly_added_articles)
                print(f"\n✅ SUCCESS: Found {len(new_articles_df)} new unique, categorized articles!")

                updated_df = pd.concat([local_df, new_articles_df], ignore_index=True)
                updated_df.sort_values(by='publishedAt', ascending=False, inplace=True)

                # <<< MODIFICATION: Ensure a clean, ordered final CSV with the new category column
                final_cols = ['category', 'title', 'source', 'publishedAt', 'description', 'link', 'image']
                # Make sure all columns exist, even if the old file didn't have them
                for col in final_cols:
                    if col not in updated_df.columns:
                        updated_df[col] = ''
                updated_df = updated_df[final_cols]

                try:
                    updated_df.to_csv(DATASET_FILE, index=False)
                    print(f"💾 Dataset '{DATASET_FILE}' successfully updated. Total articles: {len(updated_df)}")
                except Exception as e:
                    print(f"❌ CRITICAL ERROR: Could not save updated dataset! Error: {e}")
            else:
                print("\n✅ API fetch cycle complete. No new unique articles were found for the specified categories.")

            # === Step 5: Wait for the next cycle (if time permits) ===
            print("-" * 50)
            if time.time() - start_time < RUN_DURATION_SECONDS:
                 print(f"Cycle {cycle_count} complete. Waiting for {UPDATE_INTERVAL_SECONDS} seconds...")
                 time.sleep(UPDATE_INTERVAL_SECONDS)

        print("\n\n--- [ Pipeline Complete ] ---")
        print(f"Process finished automatically after the configured runtime.")

    except KeyboardInterrupt:
        print("\n\n--- [ Pipeline Shutdown ] ---")
        print("Data collection process stopped by user.")

# --- 5. EXECUTION ---
if __name__ == "__main__":
    run_data_pipeline()

### WorldWatch AI: AI Engine Training for News Categorization

In [ ]:
# Block 2: The AI Search & Optimization Engine (Complete Edition)
# Purpose: To demonstrate AI search algorithms and use a Genetic Algorithm
# to train a complete, 10-category model for use in the live dashboard.

# --- 1. SETUP ---
!pip install -q pygad
import pandas as pd
import pygad
import numpy as np
import re
import heapq
import pprint

# --- 2. LOAD & PREPARE TRAINING DATA ---
try:
    df_static = pd.read_csv('News_2025.csv')
except FileNotFoundError:
    print("Error: 'News_2025.csv' not found. Please upload it to your Colab session.")
    raise

ground_truth = {0: 'Tech', 1: 'War', 2: 'Crime/Justice', 3: 'Science', 4: 'Environment', 5: 'Economic', 6: 'War', 7: 'Peace', 8: 'Trade', 9: 'Economic', 10: 'War', 11: 'War', 12: 'Tech', 13: 'Economic', 14: 'Peace'}
sample_df = df_static.iloc[0:15].copy()
sample_df['true_category'] = sample_df.index.map(ground_truth)
print(f"Loaded and prepared training sample of {len(sample_df)} articles.")

# --- 3. UNIVERSAL CLASSIFICATION FUNCTION ---
def classify_headline(headline, keyword_db):
    headline_lower = headline.lower()
    scores = {category: 0 for category in keyword_db}
    for category, keywords in keyword_db.items():
        for keyword, score in keywords.items():
            if re.search(r'\b' + re.escape(keyword) + r'\b', headline_lower):
                scores[category] += score
    if all(s == 0 for s in scores.values()): return 'Neutral'
    # Important: Return the category with the highest ABSOLUTE score
    return max(scores, key=lambda cat: abs(scores.get(cat, 0)))

def calculate_accuracy(keyword_db):
    predictions = sample_df['title'].apply(lambda h: classify_headline(h, keyword_db))
    correct = (predictions == sample_df['true_category']).sum()
    return correct / len(sample_df)

# =====================================================================================
# SECTION A: PATHFINDING ALGORITHMS (DEMONSTRATION)
# Note: This section uses a simplified graph to demonstrate the concept of pathfinding.
# =====================================================================================
print("\n--- [Section A: Pathfinding Algorithms Demo] ---")
CANDIDATE_MODELS = {'Initial':{'war':10,'peace':-5,'economy':3}, 'High-Risk':{'war':15,'peace':-2,'economy':5}, 'Balanced':{'war':8,'peace':-8,'economy':4}, 'Peace-Focused':{'war':5,'peace':-15,'economy':1}, 'Best (Goal)':{'war':9,'peace':-7,'economy':6}}
for name, model_data in CANDIDATE_MODELS.items(): CANDIDATE_MODELS[name] = {'War':{'war': model_data['war']}, 'Peace':{'peace': model_data['peace']}, 'Economic':{'economy': model_data['economy']}}
def get_neighbors(model_name):
    connections = {'Initial': ['High-Risk','Balanced'],'High-Risk':['Initial','Best (Goal)'],'Balanced':['Initial','Peace-Focused'],'Peace-Focused':['Balanced','Best (Goal)'],'Best (Goal)':['High-Risk','Peace-Focused']}
    return connections.get(model_name, [])
def heuristic(model_name): return 1.0 - calculate_accuracy(CANDIDATE_MODELS[model_name])
def search_algorithm(start, goal, use_heuristic=False, use_path_cost=False):
    frontier = [(0, 0, [start])]; explored = set()
    while frontier:
        priority, cost, path = heapq.heappop(frontier)
        current = path[-1]
        if current in explored: continue
        explored.add(current)
        if current == goal: return path
        for neighbor in get_neighbors(current):
            if neighbor not in explored:
                new_cost = cost + 1; h = heuristic(neighbor) if use_heuristic else 0; c = new_cost if use_path_cost else 0
                heapq.heappush(frontier, (c + h, new_cost, path + [neighbor]))
    return None

print("1. Running UCS -> Path Found:", ' -> '.join(search_algorithm('Initial', 'Best (Goal)', use_path_cost=True)))
print("2. Running Greedy -> Path Found:", ' -> '.join(search_algorithm('Initial', 'Best (Goal)', use_heuristic=True)))
print("3. Running A* -> Path Found:", ' -> '.join(search_algorithm('Initial', 'Best (Goal)', use_path_cost=True, use_heuristic=True)))

# =====================================================================================
# SECTION B: LOCAL SEARCH (HILL CLIMBING - DEMONSTRATION)
# =====================================================================================
print("\n--- [Section B: Hill Climbing Demo] ---")
def hill_climbing(max_iterations=500):
    current_solution = {'War':{'war':10},'Peace':{'peace':-5},'Economic':{'economy':3}}
    current_accuracy = calculate_accuracy(current_solution)
    for _ in range(max_iterations):
        neighbor = {c:{k:v for k,v in kws.items()} for c,kws in current_solution.items()}
        cat, kw = 'War','war'
        neighbor[cat][kw] += np.random.normal(0, 1.5)
        if calculate_accuracy(neighbor) > current_accuracy:
            current_solution, current_accuracy = neighbor, calculate_accuracy(neighbor)
    return current_accuracy
print(f"1. Hill Climbing finished. Best accuracy found: {hill_climbing():.2%}")


# =====================================================================================
# SECTION C: GLOBAL OPTIMIZATION (GENETIC ALGORITHM - FULL TRAINING)
# This is the primary training engine. It uses the full, 10-category knowledge base.
# =====================================================================================
print("\n--- [Section C: Genetic Algorithm - Full Model Training] ---")

# --- THIS IS THE UPDATED, FULL KNOWLEDGE BASE ---
FULL_INITIAL_KEYWORDS = {
    'War': {'war': 10, 'attack': 9, 'conflict': 8, 'strike': 9, 'military': 7, 'invasion': 10, 'casualties': 8, 'airstrike': 9, 'troops': 6, 'battle': 8, 'weapon': 7, 'missile': 9},
    'Peace': {'peace': -5, 'treaty': -7, 'agreement': -6, 'ceasefire': -8, 'truce': -8, 'negotiation': -4, 'diplomacy': -5, 'reconciliation': -6, 'summit': -4},
    'Economic': {'economy': 3, 'inflation': 6, 'recession': 7, 'market': 4, 'growth': -2, 'gdp': 3, 'stock': 4, 'unemployment': 7, 'jobs': -3, 'fed': 5},
    'Trade': {'trade': 2, 'tariff': 5, 'sanction': 6, 'export': 1, 'import': 1, 'supply chain': 4, 'deal': -3, 'commerce': 2, 'embargo': 7},
    'Tech': {'technology': 1, 'ai': 2, 'cybersecurity': 6, 'innovation': -1, 'space': 1, 'semiconductor': 4, 'data': 3, 'software': 1, 'apple': 2},
    'Politics': {'election': 4, 'vote': 3, 'government': 3, 'congress': 4, 'president': 5, 'protest': 6, 'policy': 3, 'scandal': 6, 'impeachment': 8},
    'Health': {'health': 2, 'virus': 7, 'pandemic': 8, 'vaccine': -2, 'fda': 4, 'who': 3, 'outbreak': 7, 'disease': 6, 'hospital': 4, 'cdc': 4},
    'Environment': {'climate': 6.0, 'global warming': 7, 'emissions': 5, 'renewable': -3, 'solar': -2, 'pollution': 6, 'wildfire': 8, 'disaster': 8},
    'Crime/Justice': {'crime': 6, 'police': 4, 'arrest': 5, 'court': 4, 'trial': 5, 'lawsuit': 4, 'justice': 2, 'fbi': 5, 'prison': 5},
    'Science': {'science': 1, 'discovery': -1, 'research': 1, 'study': 1, 'nasa': 1, 'breakthrough': -2, 'experiment': 2}
}

def fitness_func_full(ga_instance, solution, solution_idx):
    temp_keywords = {}
    i = 0
    for category, keywords in FULL_INITIAL_KEYWORDS.items():
        temp_keywords[category] = {}
        for keyword in keywords:
            temp_keywords[category][keyword] = solution[i]
            i += 1
    return calculate_accuracy(temp_keywords)

initial_weights = [score for cat_kw in FULL_INITIAL_KEYWORDS.values() for score in cat_kw.values()]
ga_instance = pygad.GA(num_generations=100, num_parents_mating=7, fitness_func=fitness_func_full,
                       sol_per_pop=20, num_genes=len(initial_weights), init_range_low=-15.0,
                       init_range_high=15.0, parent_selection_type="sss", keep_parents=2,
                       crossover_type="single_point", mutation_type="random", mutation_percent_genes=8,
                       suppress_warnings=True)

print("1. Running Genetic Algorithm to find optimal weights for the FULL 10-category model...")
ga_instance.run()
best_solution, best_solution_fitness, _ = ga_instance.best_solution()
print(f"   - Optimization complete. Best accuracy on sample data: {best_solution_fitness:.2%}")

# --- 7. FINAL OUTPUT ---
OPTIMIZED_KEYWORDS = {}
i = 0
for category, keywords in FULL_INITIAL_KEYWORDS.items():
    OPTIMIZED_KEYWORDS[category] = {}
    for keyword in keywords:
        OPTIMIZED_KEYWORDS[category][keyword] = best_solution[i]
        i += 1

print("\n\n" + "="*60)
print("--- ENGINE COMPLETE: OPTIMIZED AI MODEL GENERATED ---")
print("--- Copy the dictionary below and paste it into Block 2 ---")
print("="*60)
pprint.pprint(OPTIMIZED_KEYWORDS)

### WorldWatch AI Dashboard: Live Categorized News View

In [ ]:
# Block 3: WorldWatch AI Dashboard (V21.0 - High-Precision Final Edition)
# Purpose: The definitive dashboard with highly-targeted, contextual queries to
# ensure the "War" and "Economic" feeds are clean, relevant, and professional.

# --- 1. SETUP ---
import pandas as pd
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
from IPython.display import display, HTML, clear_output
import re
from datetime import datetime
import os
import time
import requests
from urllib.parse import quote

# --- 2. CONFIGURATION & AI MODELS ---

# --- AI Model for Classification ---
OPTIMIZED_KEYWORDS = {
    'Economic': {'bonds': -4.56, 'budget': 7.63, 'downturn': 14.28, 'economy': 11.23, 'fed': 13.78, 'gdp': 9.22, 'growth': 8.52, 'inflation': 14.88, 'jobs': 12.01, 'market': 12.07, 'rates': 14.93, 'recession': 14.89, 'stimulus': 11.19, 'stock': 12.98, 'unemployment': 14.53},
    'Peace': {'accord': 8.84, 'agreement': 14.15, 'ceasefire': 14.93, 'de-escalation': 14.39, 'diplomacy': 14.99, 'diplomatic': 14.97, 'mediation': 4.13, 'negotiation': 14.89, 'peace': 15.0, 'summit': 14.92, 'treaty': 14.97, 'truce': 14.88, 'un': -4.27},
    'Tech': {'ai': 14.24, 'algorithm': 9.0, 'amazon': -0.11, 'apple': 14.82, 'cybersecurity': 14.97, 'data': 12.18, 'google': 14.96, 'innovation': 14.11, 'launch': 8.32, 'meta': 14.49, 'metaverse': 14.77, 'microsoft': 14.91, 'nvidia': 14.92, 'semiconductor': 14.95, 'software': 12.01, 'tech': 14.86},
    'Trade': {'commerce': 14.98, 'deal': 14.95, 'embargo': 14.12, 'export': 14.13, 'goods': 14.22, 'import': 14.83, 'logistics': 12.0, 'ports': 10.39, 'sanction': 14.93, 'shipping': 11.33, 'supply chain': 14.99, 'tariff': 14.99, 'trade': 14.99, 'wto': 8.21},
    'War': {'airstrike': 14.95, 'artillery': 14.89, 'attack': 14.95, 'battle': 14.91, 'casualties': 15.0, 'conflict': 14.99, 'defense': 14.92, 'drone': 14.54, 'escalation': 14.8, 'geopolitical': 14.42, 'invasion': 14.97, 'military': 14.98, 'missile': 14.99, 'offensive': 14.86, 'strike': 15.0, 'troops': 14.98, 'weapon': 14.85}
}

# --- Data Collection Pipeline Configuration ---
# <<< MODIFICATION: This is the definitive correction for your entire project.
# These queries are now highly specific and contextual.
CATEGORIZED_QUERIES = {
    # War: Must mention a specific actor AND a specific action.
    "War": '("Israel" OR "Iran" OR "USA" OR "Gaza" OR "Hezbollah" OR "Palestine") AND (attack OR war OR strike OR missile OR conflict OR airstrike OR invasion OR offensive)',

    # Economic: Must mention an economic term AND be in the context of the conflict.
    "Economic": '("economy" OR "inflation" OR "oil prices" OR "supply chain" OR "stock market" OR "recession" OR "sanctions") AND ("Israel" OR "Iran" OR "Gaza" OR "Middle East")',

    # Peace: Remains broad but focused on diplomatic terms.
    "Peace": "peace talks OR treaty OR diplomacy OR ceasefire OR negotiations OR UN resolution",

    # Trade & Tech: Remain broad as they are less prone to irrelevant noise.
    "Trade": "trade deal OR tariff OR supply chain OR exports OR imports OR wto",
    "Tech": "AI OR artificial intelligence OR semiconductor OR cybersecurity OR tech giant"
}
MY_API_KEY = "1ddd3a86143c475cb371f487a3617cbc"

# --- UI Configuration ---
COLORS = {
    'War': '#ff6b6b', 'Peace': '#63e6be', 'Economic': '#fcc419', 'Trade': '#ff922b', 'Tech': '#4dabf7', 'Neutral': '#adb5bd',
    'background': '#f8f9fa', 'card_bg': '#ffffff', 'text_primary': '#212529', 'text_secondary': '#868e96',
    'shadow': 'rgba(0, 0, 0, 0.05)', 'border': '#dee2e6',
    'button_primary': '#A7D9E0', 'button_primary_hover': '#97C8CF',
    'header_bg_start': '#CDECF2', 'header_bg_end': '#A7D9E0',
    'header_text_main': '#014a60', 'header_text_sub': '#027899',
    'status_info_bg': '#e7f5ff', 'status_info_border': '#a5d8ff', 'status_info_text': '#1c7ed6',
    'status_success_bg': '#e6fcf5', 'status_success_border': '#b2f2bb', 'status_success_text': '#2f9e44',
    'status_error_bg': '#fff5f5', 'status_error_border': '#ffc9c9', 'status_error_text': '#e03131'
}
TAB_CATEGORIES = {'All Headlines': 'globe','War': 'fighter-jet','Peace': 'handshake-o','Economic': 'line-chart','Trade': 'exchange','Tech': 'cogs'}
FINAL_DF_SCHEMA = ['title', 'source', 'publishedAt', 'link', 'image', 'category', 'score']
DATASET_FILE = 'News_2025.csv'


# --- 3. DASHBOARD & PIPELINE FUNCTIONS ---
# (The Python logic functions from here are unchanged as they adapt to the new data)
status_area = widgets.Output()

def show_status_message(message, style_class, icon):
    status_area.clear_output(wait=True)
    with status_area:
        display(HTML(f"""<div class="status-banner {style_class}"><i class="fa {icon} status-icon"></i><span class="status-message">{message}</span></div>"""))

def fetch_live_news_from_url(api_key, url):
    headers = {'X-Api-Key': api_key}
    try:
        response = requests.get(url, headers=headers, timeout=15)
        if response.status_code == 200:
            articles = response.json().get('articles', [])
            if not articles: return pd.DataFrame()
            df = pd.DataFrame(articles)
            df.rename(columns={'url': 'link', 'urlToImage': 'image'}, inplace=True)
            df['source'] = df['source'].apply(lambda s: s.get('name') if isinstance(s, dict) else s)
            required_cols = ['title', 'source', 'publishedAt', 'description', 'link', 'image']
            for col in required_cols:
                if col not in df.columns: df[col] = ''
            return df.dropna(subset=['title', 'publishedAt'])[required_cols]
        return None
    except requests.exceptions.RequestException: return None

def run_live_data_collection():
    show_status_message("Synchronizing with high-precision news feeds...", "status-info", "fa-hourglass-half")
    time.sleep(1)
    try:
        local_df = pd.read_csv(DATASET_FILE) if os.path.exists(DATASET_FILE) else pd.DataFrame()
        existing_titles = set(local_df['title']) if 'title' in local_df.columns else set()
        live_articles_dfs = []
        for category, query in CATEGORIZED_QUERIES.items():
            encoded_query = quote(query)
            url = f"https://newsapi.org/v2/everything?q={encoded_query}&language=en&sortBy=publishedAt"
            new_df = fetch_live_news_from_url(MY_API_KEY, url)
            if new_df is not None and not new_df.empty:
                new_df['category'] = category
                live_articles_dfs.append(new_df)
            time.sleep(0.5)
        if not live_articles_dfs:
            show_status_message("No new articles were found matching the specific criteria. Feeds are monitored.", "status-info", "fa-info-circle")
            return
        live_df_combined = pd.concat(live_articles_dfs, ignore_index=True).drop_duplicates(subset=['title'], keep='first')
        newly_added_articles = [article for _, article in live_df_combined.iterrows() if article['title'] not in existing_titles]
        if newly_added_articles:
            show_status_message(f"Found and added {len(newly_added_articles)} new, highly relevant articles.", "status-success", "fa-check-circle")
            updated_df = pd.concat([local_df, pd.DataFrame(newly_added_articles)], ignore_index=True)
            updated_df.sort_values(by='publishedAt', ascending=False, inplace=True).to_csv(DATASET_FILE, index=False)
        else:
            show_status_message("All feeds checked. Your dataset is already up-to-date with the latest intelligence.", "status-success", "fa-check-circle")
        time.sleep(1.5)
    except Exception as e:
        show_status_message(f"An error occurred during data collection: {e}", "status-error", "fa-exclamation-triangle")

def classify_and_score(headline, keyword_db):
    headline_lower, scores = str(headline).lower(), {category: 0 for category in keyword_db}
    for category, keywords in keyword_db.items():
        for keyword, score in keywords.items():
            if re.search(r'\b' + re.escape(keyword) + r'\b', headline_lower): scores[category] += score
    if all(s == 0 for s in scores.values()): return 'Neutral', 0
    top_category = max(scores, key=scores.get)
    return top_category, round(scores[top_category], 1)

def render_article_card(article):
    category, color = article.get('category', 'Neutral'), COLORS.get(article.get('category', 'Neutral'), COLORS['Neutral'])
    image_url = article.get('image', '')
    img_html = f'<div class="card-image-container"><img class="card-image" src="{image_url}" alt="Image not available" onerror="this.parentElement.classList.add(\'no-image\');"></div>' if image_url and isinstance(image_url, str) else '<div class="card-image-container no-image"></div>'
    return f"""<a href="{article['link']}" target="_blank" class="article-card-link"><div class="article-card" style="border-left-color: {color};">{img_html}<div class="card-content"><div class="card-category-tag" style="background-color: {color};">{category.upper()}</div><h3 class="card-title">{article['title']}</h3><div class="card-footer"><span class="card-source">{article.get('source', 'Unknown Source')}</span><span class="card-published">{article['publishedAt'].strftime('%b %d, %Y')}</span></div></div></div></a>"""

def create_content_view(df, status_message=""):
    if df.empty: return f"""<div class="empty-state"><i class="fa fa-search" style="font-size: 4em; color: #ced4da;"></i><h2>{status_message}</h2><p>No articles match the current filter. Try clicking Refresh.</p></div>"""
    cards_html = "".join([render_article_card(row) for _, row in df.iterrows()])
    return f"<div class='articles-grid'>{cards_html}</div>"

master_df, active_tab_name, output_area = pd.DataFrame(columns=FINAL_DF_SCHEMA), 'All Headlines', widgets.Output()
def update_display_area(clear_status=False):
    output_area.clear_output(wait=True)
    if clear_status: status_area.clear_output()
    query = search_input.value.strip().lower()
    df_to_process = master_df
    if query: df_to_process = master_df[master_df['title'].str.lower().str.contains(query, na=False)]
    if active_tab_name == 'All Headlines':
        priority_categories = ['War', 'Economic', 'Tech']
        df_for_view = df_to_process[df_to_process['category'].isin(priority_categories)]
        status_msg = f"Showing {len(df_for_view)} priority articles"
        empty_msg = "No articles found for War, Economic, or Tech."
    else:
        df_for_view = df_to_process[df_to_process['category'] == active_tab_name]
        status_msg = f"Showing {len(df_for_view)} articles in '{active_tab_name}'"
        empty_msg = f"No articles found for '{active_tab_name}'"
    content_html = create_content_view(df_for_view, empty_msg)
    footer = f"""<div class="dashboard-footer"><p>WorldWatch AI Live Edition</p><p class="footer-separator">|</p><p>{status_msg}</p><p class="footer-separator">|</p><p>Last update: {datetime.now().strftime('%H:%M:%S')}</p></div>"""
    with output_area: display(HTML(f"<div class='fade-in'>{content_html}{footer}</div>"))

def handle_refresh_click(b=None):
    global master_df, active_tab_name
    output_area.clear_output(wait=True)
    with output_area: display(HTML('<div class="loader-container"><div class="loader"></div><h3>Preparing Dashboard...</h3></div>'))
    run_live_data_collection()
    if not os.path.exists(DATASET_FILE):
        master_df = pd.DataFrame(columns=FINAL_DF_SCHEMA)
        update_display_area(clear_status=True)
        return
    try:
        temp_df = pd.read_csv(DATASET_FILE)
        temp_df['link'] = temp_df['link'].fillna('')
        temp_df['publishedAt'] = pd.to_datetime(temp_df['publishedAt'], errors='coerce', utc=True).dt.tz_localize(None)
        temp_df.dropna(subset=['title', 'publishedAt'], inplace=True)
        results = temp_df['title'].apply(lambda h: classify_and_score(h, OPTIMIZED_KEYWORDS)).tolist()
        temp_df[['category', 'score']] = pd.DataFrame(results, index=temp_df.index)
        master_df = temp_df[FINAL_DF_SCHEMA].copy().sort_values(by='publishedAt', ascending=False)
        search_input.value, active_tab_name = '', 'All Headlines'
        for btn in tab_buttons: btn.button_style = 'info' if btn.description == active_tab_name else ''
        update_display_area()
    except Exception as e:
        show_status_message(f"A critical error occurred while processing the dataset: {e}", "status-error", "fa-exclamation-triangle")

def on_filter_change(b):
    global active_tab_name
    if isinstance(b, widgets.Button) and b.description in TAB_CATEGORIES:
        active_tab_name = b.description
        for btn in tab_buttons: btn.button_style = 'info' if btn.description == active_tab_name else ''
    update_display_area(clear_status=True)

# --- 5. UI LAYOUT ---
global_css = widgets.HTML(f"""<style>@import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap');
:root {{ --main-font: 'Inter', sans-serif; --bg: {COLORS['background']}; --card-bg: {COLORS['card_bg']}; --text1: {COLORS['text_primary']}; --text2: {COLORS['text_secondary']}; --border: {COLORS['border']}; --shadow: {COLORS['shadow']}; }}
.dashboard-header {{ background: linear-gradient(135deg, {COLORS['header_bg_start']} 0%, {COLORS['header_bg_end']} 100%); border-bottom: 1px solid {COLORS['button_primary_hover']}; padding: 30px 20px; text-align: center; border-radius: 12px; margin-bottom: 20px; box-shadow: 0 4px 15px rgba(0, 0, 0, 0.05);}}
.dashboard-header h1 {{ margin: 0; font-size: 2.5em; font-weight: 700; color: {COLORS['header_text_main']}; }}
.dashboard-header p {{ font-size: 1.1em; margin-top: 5px; color: {COLORS['header_text_sub']}; font-weight: 500; }}
.jupyter-button.mod-primary {{ background-color: {COLORS['button_primary']} !important; border: 1px solid {COLORS['button_primary_hover']} !important; color: #212529 !important; }}
.jupyter-button.mod-primary:hover {{ background-color: {COLORS['button_primary_hover']} !important; }}
.jupyter-widgets.widget-text input[type="text"]:focus {{ border-color: {COLORS['Tech']}; box-shadow: 0 0 0 3px {COLORS['Tech']}33; }}
.articles-grid {{ display: grid; grid-template-columns: repeat(auto-fill, minmax(320px, 1fr)); gap: 25px; }}
.article-card-link {{ text-decoration: none; display: flex; }}
.article-card {{ background: var(--card-bg); border-radius: 12px; box-shadow: 0 4px 20px var(--shadow); border-left: 5px solid; transition: all 0.25s ease; display: flex; flex-direction: column; width: 100%; }}
.article-card:hover {{ transform: translateY(-5px); box-shadow: 0 8px 25px rgba(0,0,0,0.08); }}
.card-image-container {{ width: 100%; aspect-ratio: 16 / 9; background-color: #f1f3f5; border-radius: 10px 10px 0 0; overflow: hidden; }}
.card-image-container.no-image {{ background-image: url('https://www.gstatic.com/images/icons/material/system/2x/photo_camera_grey600_48dp.png'); background-size: 48px; background-position: center; background-repeat: no-repeat; }}
.card-image {{ width: 100%; height: 100%; object-fit: cover; }}
.card-content {{ padding: 20px; display: flex; flex-direction: column; flex-grow: 1; }}
.card-category-tag {{ padding: 4px 10px; border-radius: 6px; font-weight: 600; color: white; font-size: 0.75em; display: inline-block; margin-bottom: 12px; }}
.card-title {{ font-size: 1.1em; font-weight: 600; color: var(--text1); margin: 0 0 15px 0; line-height: 1.45; flex-grow: 1; }}
.card-footer {{ display: flex; justify-content: space-between; align-items: center; font-size: 0.85em; color: var(--text2); border-top: 1px solid var(--border); padding-top: 15px; margin-top: auto; }}
.loader {{ height: 50px; width: 50px; margin: 0 auto 20px auto; border: 5px solid #e9ecef; border-top-color: {COLORS['Tech']}; border-radius: 50%; animation: spin 1s linear infinite; }} @keyframes spin {{ to {{ transform: rotate(360deg); }} }}
.status-banner {{ padding: 15px 20px; margin: 0 0 25px 0; border: 1px solid; border-radius: 8px; display: flex; align-items: center; animation: fadeInDown 0.5s ease-out; }}
.status-icon {{ font-size: 1.4em; margin-right: 15px; }}
.status-message {{ font-size: 1em; font-weight: 500; }}
.status-info {{ background-color: {COLORS['status_info_bg']}; border-color: {COLORS['status_info_border']}; color: {COLORS['status_info_text']}; }}
.status-success {{ background-color: {COLORS['status_success_bg']}; border-color: {COLORS['status_success_border']}; color: {COLORS['status_success_text']}; }}
.status-error {{ background-color: {COLORS['status_error_bg']}; border-color: {COLORS['status_error_border']}; color: {COLORS['status_error_text']}; }}
@keyframes fadeInDown {{ from {{ opacity: 0; transform: translateY(-10px); }} to {{ opacity: 1; transform: translateY(0); }} }}
.dashboard-footer {{ display: flex; justify-content: center; align-items: center; gap: 15px; padding: 20px; color: var(--text2); font-size: 0.9em; border-top: 1px solid var(--border); margin-top: 30px; }}
.footer-separator {{ color: var(--border); }}
</style>""")
header = widgets.HTML(f"""<div class="dashboard-header"><h1>WorldWatch AI</h1><p>Your Global Intelligence Command Center</p></div>""")
refresh_button = widgets.Button(description="Fetch & Refresh Data", icon='refresh', button_style='primary', layout=Layout(width='auto', margin='0 15px 0 0'))
search_input = widgets.Text(placeholder='Search articles...', layout=Layout(width='auto', flex='1 1 0'))
search_button = widgets.Button(icon='search', layout=Layout(width='auto'))
tab_buttons = [widgets.Button(description=t, icon=i, layout=Layout(min_width='90px', width='auto', margin='2px')) for t,i in TAB_CATEGORIES.items()]
if tab_buttons: tab_buttons[0].button_style = 'info'
refresh_button.on_click(handle_refresh_click)
search_input.on_submit(on_filter_change)
search_button.on_click(on_filter_change)
for btn in tab_buttons: btn.on_click(on_filter_change)
top_controls = HBox([refresh_button, search_input, search_button], layout=Layout(width='100%', justify_content='center', align_items='center'))
control_panel = VBox([top_controls, HBox(tab_buttons, layout=Layout(justify_content='center', margin='15px 0 0 0', flex_wrap='wrap'))], layout=Layout(width='100%', margin='0 0 20px 0'))
app_layout = VBox([global_css, header, control_panel, status_area, output_area], layout=Layout(width='98%', max_width='1400px', margin='20px auto'))

# --- 6. LAUNCH DASHBOARD ---
display(app_layout)
handle_refresh_click()